In [9]:
import numpy as np
import tensorflow as tf 

import os
from sklearn.model_selection import train_test_split
import csv

with open('data/driving_log.csv') as f:
    x = [line for line in csv.reader(f)]
x_train, x_val = train_test_split(x, test_size=0.05)

import matplotlib.image as mpimg
from sklearn.utils import shuffle

PREFIX = './data/';

def generator(x, batch):
    while True:
        shuffle(x)
        for offset in range(0, len(x), batch):
            img_list = []
            metrics_list = []
            for xx in x[offset:offset+batch]:
                steering_center = float(xx[3])
                correction = 0.2 
                steering_left = steering_center + correction
                steering_right = steering_center - correction
                for i in range(3):
                    img = mpimg.imread(PREFIX + xx[i].strip())
                    img_list.append(img)
                metrics_list += [steering_center, steering_left, steering_right]

            yield shuffle(np.array(img_list), np.array(metrics_list))

from keras.layers import Dense, Dropout, Activation, Flatten, ELU
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Cropping2D
from keras.optimizers import Adam
from keras.layers.core import Lambda

CROPPING_DIMS = ((70, 25), (0, 0))
INPUT_SHAPE = (160, 320, 3)
CONV_1 = ((16, 8, 8))
CONV_2 = (32, 5, 5)
CONV_3 = (64, 5, 5)
RESIZE_DIMS = [40, 160]
LEARNING_RATE = 0.0001
FULLY_CONNECTED_LAYER1 = 512
FULLY_CONNECTED_LAYER2 = 50
FULLY_CONNECTED_LAYER3 = 1
DROPOUT = 0.5

def resize(img):
    import tensorflow as tf
    return tf.image.resize_images(img, RESIZE_DIMS)

model = Sequential()

#crop
model.add(Cropping2D(cropping=CROPPING_DIMS, 
                     dim_ordering='tf',  
                     input_shape=INPUT_SHAPE)) 

# normalize
model.add(Lambda(lambda x: (x/255.0) - 0.5))

# add convolutional layers 
model.add(Convolution2D(*CONV_1, subsample=(4, 4), border_mode="same")) 
model.add(ELU()) 
model.add(Convolution2D(*CONV_2, subsample=(2, 2), border_mode="same")) 
model.add(ELU()) 
model.add(Convolution2D(*CONV_3, subsample=(2, 2), border_mode="same")) 
model.add(Flatten()) 
model.add(ELU()) 

# add fully connected layers
model.add(Dense(FULLY_CONNECTED_LAYER1)) 
model.add(Dropout(DROPOUT)) 
model.add(ELU()) 
model.add(Dense(FULLY_CONNECTED_LAYER2)) 
model.add(ELU()) 
model.add(Dense(FULLY_CONNECTED_LAYER3)) 

# compile
model.compile(optimizer=Adam(lr=LEARNING_RATE), loss="mse", metrics=['accuracy'])
model.summary()
model.fit_generator(generator(x_train, 32), 
                    samples_per_epoch=len(x_train) * 3, 
                    validation_data=generator(x_val, 32),
                    nb_val_samples=len(x_val), nb_epoch=5)
model.save('model.h5')

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cropping2d_4 (Cropping2D)        (None, 65, 320, 3)    0           cropping2d_input_3[0][0]         
____________________________________________________________________________________________________
lambda_4 (Lambda)                (None, 65, 320, 3)    0           cropping2d_4[0][0]               
____________________________________________________________________________________________________
convolution2d_7 (Convolution2D)  (None, 17, 80, 16)    3088        lambda_4[0][0]                   
____________________________________________________________________________________________________
elu_11 (ELU)                     (None, 17, 80, 16)    0           convolution2d_7[0][0]            
___________________________________________________________________________________________

KeyboardInterrupt: 